In [1]:
import os
import time
import numpy as np
from astropy import units
from astropy.coordinates import Angle

from pyuvdata import UVData

from bda import bda_tools
import importlib
importlib.reload(bda_tools)

<module 'bda.bda_tools' from '/Users/bryna/anaconda/envs/py36/lib/python3.6/site-packages/bda/bda_tools.py'>

In [2]:
def apply_bda(file_in, file_out, pre_fs_int_time=0.1, corr_FoV_angle=20.0,
              max_time=30, corr_int_time=None, max_decorr=0.1, overwrite=False, filetype='uvh5'):

    if os.path.exists(file_out) and overwrite is False:
        raise ValueError("{} exists. Use --overwrite to overwrite the file.".format(file_out))

    # check that output filetype is valid
    if filetype not in ("uvh5", "uvfits", "miriad"):
        raise ValueError("filetype must be one of uvh5, uvfits, or miriad")

    uv = UVData()
    uv.read(file_in)

    # apply BDA
    pre_fs_int_time = pre_fs_int_time * units.s
    corr_FoV_angle = Angle(corr_FoV_angle, units.deg)
    max_time = max_time * units.s
    uv2 = bda_tools.apply_bda(
        uv,
        max_decorr,
        pre_fs_int_time,
        corr_FoV_angle,
        max_time,
        corr_int_time,
    )

    # write out file
    if filetype == "uvh5":
        uv2.write_uvh5(file_out, clobber=True)
    if filetype == "uvfits":
        uv2.write_uvfits(file_out, spoof_nonessential=True, force_phase=True)
    if filetype == "miriad":
        uv2.write_miriad(file_out, clobber=True)


In [3]:
uvfits_file_in = '/Volumes/Data1/pyuvsim_ref_sim/simulation_results/ref_1.2_uniform.uvfits'
uvh5_file_in = '/Volumes/Data1/pyuvsim_ref_sim/simulation_results/ref_1.2_uniform.uvh5'
uvh5_file_in_fixed = '/Volumes/Data1/pyuvsim_ref_sim/simulation_results/ref_1.2_uniform_fixed.uvh5'
if not os.path.isfile(uvh5_file_in_fixed):
    t0 = time.time()
    uv_in = UVData()
    print_str = 'time to read in '
    if not os.path.isfile(uvh5_file_in):
        uv_in.read(uvfits_file_in)
        print_str += 'uvfits file '
    else:
        uv_in.read(uvh5_file_in)
        print_str += 'uvh5 file '
        
    if min(uv_in.integration_time) < 0.9:
        # simulation files were originally written with int time in days not seconds
        uv_in.integration_time = uv_in.integration_time * 3600 * 24
        uv_in.set_lsts_from_time_array()

    if uv_in.phase_type == 'phased':
        uv_in.unphase_to_drift()
        
    uv_in.write_uvh5(uvh5_file_in_fixed)
    t1 = time.time()
    
    print_str += 'and fix it: {t} seconds'.format(t=t1-t0)
    print(print_str)

Telescope BLLITE is not in known_telescopes.


time to read in uvh5 file and fix it: 2680.3350892066956 seconds


In [4]:
file_in=uvh5_file_in_fixed
file_out='/Volumes/Data1/bda_outputs/ref_1.2_uniform_fixed_bda.uvh5'
initial_int_time=1.0  # seconds

In [5]:
if not os.path.isfile(file_out):
    t0 = time.time()
    apply_bda(file_in, file_out, pre_fs_int_time=initial_int_time, corr_FoV_angle=20.0,
              max_time=30, corr_int_time=None, overwrite=False, filetype='uvh5')
    t1 = time.time()
    print('time to read file into a uvdata object and do bda: {t} seconds'.format(t=t1-t0))

In [6]:
uv_in = UVData()
uv_in.read(file_in)
uv_bda = UVData()
uv_bda.read(file_out)


Telescope BLLITE is not in known_telescopes.
LST values stored in /Volumes/Data1/bda_outputs/ref_1.2_uniform_fixed_bda.uvh5 are not self-consistent with time_array and telescope location. Consider recomputing with utils.get_lst_for_time.


In [7]:
print('Input data')
print('Ntimes: ', uv_in.Ntimes)
print('Nfreqs: ', uv_in.Nfreqs)
print('Nbls: ', uv_in.Nbls)
print('Nblts: ', uv_in.Nblts)
print('data_array shape: ', uv_in.data_array.shape)
print('phase_type: ', uv_in.phase_type)

Input data
Ntimes:  86400
Nfreqs:  1
Nbls:  10
Nblts:  864000
data_array shape:  (864000, 1, 1, 4)
phase_type:  drift


In [8]:
print('BDA data')
print('Ntimes: ', uv_bda.Ntimes)
print('Nfreqs: ', uv_bda.Nfreqs)
print('Nbls: ', uv_bda.Nbls)
print('Nblts: ', uv_bda.Nblts)
print('data_array shape: ', uv_bda.data_array.shape)
print('phase_type: ', uv_in.phase_type)

BDA data
Ntimes:  117818
Nfreqs:  1
Nbls:  10
Nblts:  334634
data_array shape:  (334634, 1, 1, 4)
phase_type:  drift


In [9]:
unique_bls = np.unique(uv_in.baseline_array)

In [10]:
np.unique(uv_in.integration_time)

array([0.99999458])

In [11]:
np.unique(uv_in.integration_time)*3600*24

array([86399.53136444])

In [12]:
for bl in unique_bls:
    print('baseline: ', bl)
    time_diffs = np.diff(np.sort(uv_in.time_array[np.where(uv_in.baseline_array == bl)]))*24*60*60
    int_times = uv_in.integration_time[np.where(uv_in.baseline_array == bl)]
    nsamples = uv_in.nsample_array[np.where(uv_in.baseline_array == bl)]
    print('min int_times: ', np.min(int_times))
    print('max int_times: ', np.max(int_times))
    print('min time_diffs: ', np.min(time_diffs))
    print('max time_diffs: ', np.max(time_diffs))

baseline:  67585
min int_times:  0.9999945759773254
max int_times:  0.9999945759773254
min time_diffs:  0.9990692138671875
max time_diffs:  1.0042190551757812
baseline:  69633
min int_times:  0.9999945759773254
max int_times:  0.9999945759773254
min time_diffs:  0.9990692138671875
max time_diffs:  1.0042190551757812
baseline:  69634
min int_times:  0.9999945759773254
max int_times:  0.9999945759773254
min time_diffs:  0.9990692138671875
max time_diffs:  1.0042190551757812
baseline:  71681
min int_times:  0.9999945759773254
max int_times:  0.9999945759773254
min time_diffs:  0.9990692138671875
max time_diffs:  1.0042190551757812
baseline:  71682
min int_times:  0.9999945759773254
max int_times:  0.9999945759773254
min time_diffs:  0.9990692138671875
max time_diffs:  1.0042190551757812
baseline:  71683
min int_times:  0.9999945759773254
max int_times:  0.9999945759773254
min time_diffs:  0.9990692138671875
max time_diffs:  1.0042190551757812
baseline:  73729
min int_times:  0.99999457597

In [13]:
for bl in unique_bls:
    print('baseline: ', bl)
    time_diffs = np.diff(np.sort(uv_bda.time_array[np.where(uv_bda.baseline_array == bl)]))*24*60*60
    int_times = uv_bda.integration_time[np.where(uv_bda.baseline_array == bl)]
    nsamples = uv_bda.nsample_array[np.where(uv_bda.baseline_array == bl)]
    print('min int_times: ', np.min(int_times))
    print('max int_times: ', np.max(int_times))
    print('min time_diffs: ', np.min(time_diffs))
    print('max time_diffs: ', np.max(time_diffs))

baseline:  67585
min int_times:  0.0
max int_times:  0.0
min time_diffs:  0.9990692138671875
max time_diffs:  1.0042190551757812
baseline:  69633
min int_times:  7.9999566078186035
max int_times:  7.9999566078186035
min time_diffs:  8.999347686767578
max time_diffs:  9.001922607421875
baseline:  69634
min int_times:  0.0
max int_times:  0.0
min time_diffs:  0.9990692138671875
max time_diffs:  1.0042190551757812
baseline:  71681
min int_times:  3.9999783039093018
max int_times:  7.9999566078186035
min time_diffs:  7.001209259033203
max time_diffs:  9.001922607421875
baseline:  71682
min int_times:  3.9999783039093018
max int_times:  6.999962031841278
min time_diffs:  6.4990997314453125
max time_diffs:  8.002853393554688
baseline:  71683
min int_times:  0.0
max int_times:  0.0
min time_diffs:  0.9990692138671875
max time_diffs:  1.0042190551757812
baseline:  73729
min int_times:  0.9999945759773254
max int_times:  7.9999566078186035
min time_diffs:  5.500030517578125
max time_diffs:  9.0